In [7]:
import random
from torchvision import datasets, transforms
import torch.optim as optim
import torch.nn as nn
import pandas as pd
import numpy as np
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.utils.data as Data
import matplotlib.pyplot as plt
import math
import scienceplots
import matplotlib as mpl
from tqdm import tqdm
import optuna

In [8]:

SEED = 1234
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)
np.random.seed(SEED)
random.seed(SEED)
torch.backends.cudnn.deterministic = True

# 定义数据
x = torch.unsqueeze(torch.linspace(-2, 10, 19), dim=1)
ytrain = np.exp(-(x - 2) ** 2) + np.exp(-(x - 6) ** 2 / 10) + 1 / (x ** 2 + 1) + 0.12 * torch.randn(x.size())


class CustomActivation(nn.Module):  # 激活函数含噪声参数
    def __init__(self, trial):
        super().__init__()
        self.sigma = trial.suggest_float("sigma", 0, 10)  # 1个可学习参数 sigma

    def forward(self, input):
        temp = 1/2 + torch.erf(input / (math.sqrt(2) * self.sigma))/2  # 前向传播
        return temp


# 定义网络
class Net(nn.Module):
    def __init__(self, n_feature, n_hidden1, n_output, trial):
        super(Net, self).__init__()
        self.hidden1 = nn.Linear(n_feature, n_hidden1)  # 全连接层
        self.custom1 = CustomActivation(trial)  # 自定义激活函数层
        self.predict = nn.Linear(n_hidden1, n_output)  # 输出层

    def forward(self, x):
        x = self.hidden1(x)
        x = self.custom1(x)
        x = self.predict(x)  # 前向传播过程
        return x

    def reset_parameters(self):
        self.hidden1.reset_parameters()
        self.predict.reset_parameters()


device = torch.device('cpu')

In [9]:


def objective(trial):
    net = Net(n_feature=1, n_hidden1=14, n_output=1, trial=trial).to(device)
    optimizer = optim.Adam(net.parameters(), lr=0.01, betas=(0.99, 0.99))
    loss_func = torch.nn.MSELoss().to(device)
    NJnum = 1
    epoch = 15000
    train_losses = []

    for num in range(NJnum):
        seed = random.sample(range(1, 1000), 1)[0]  # 选择一个随机种子
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)

        net.reset_parameters()  # 每次实验前重置网络参数

        # 训练网络
        for _ in range(epoch):
            net.train()
            prediction = net(x)
            loss = loss_func(prediction, ytrain)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        train_losses.append(loss.item())  # 记录本次训练的损失值

    train_loss = np.mean(train_losses)  # 计算平均训练损失

    # 测试阶段
    seeds = random.sample(range(1, 1000), 10)
    test_losses = []

    for seed in seeds:
        torch.manual_seed(seed)
        np.random.seed(seed)
        random.seed(seed)
        x_test = torch.unsqueeze(torch.linspace(-2, 10, 30), dim=1)
        y_test = np.exp(-(x_test - 2) ** 2) + np.exp(-(x_test - 6) ** 2 / 10) + 1 / (
                    x_test ** 2 + 1) + 0.12 * torch.randn(x_test.size())

        net.eval()
        with torch.no_grad():
            prediction_test = net(x_test)
            test_loss = loss_func(prediction_test, y_test)
            test_losses.append(test_loss.item())

    mean_test_loss = np.mean(test_losses)
    return mean_test_loss


study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

[I 2024-10-11 14:43:56,659] A new study created in memory with name: no-name-7ab59e92-ebd4-4d22-87fc-3e58410ede7b
[I 2024-10-11 14:44:30,201] Trial 0 finished with value: 0.018781430553644896 and parameters: {'sigma': 8.896000845353173}. Best is trial 0 with value: 0.018781430553644896.
[I 2024-10-11 14:45:11,956] Trial 1 finished with value: 0.02101924056187272 and parameters: {'sigma': 1.323511228766291}. Best is trial 0 with value: 0.018781430553644896.
[I 2024-10-11 14:45:47,085] Trial 2 finished with value: 0.025938696786761285 and parameters: {'sigma': 1.7566905278451461}. Best is trial 0 with value: 0.018781430553644896.
[I 2024-10-11 14:46:08,507] Trial 3 finished with value: 0.018425903655588625 and parameters: {'sigma': 4.067359844727557}. Best is trial 3 with value: 0.018425903655588625.
[I 2024-10-11 14:46:28,835] Trial 4 finished with value: 0.01987476581707597 and parameters: {'sigma': 2.9341892695444027}. Best is trial 3 with value: 0.018425903655588625.
[I 2024-10-11 14

In [10]:

# 获取最佳 sigma 值
best_sigma = study.best_params["sigma"]
print("Best sigma:", best_sigma)

Best sigma: 5.6566977648877765


In [14]:
class CustomActivation(nn.Module):  # 激活函数含噪声参数
    def __init__(self, sigma):
        super().__init__()
        self.sigma = sigma

    def forward(self, input):
        temp = 1/2 + torch.erf(input / (math.sqrt(2) * self.sigma))/2   # 前向传播
        return temp


class Net(nn.Module):  # 定义网络
    def __init__(self, n_feature, n_hidden1, n_output, sigma):
        super(Net, self).__init__()
        self.hidden1 = nn.Linear(n_feature, n_hidden1)  # 全连接层
        self.custom1 = CustomActivation(sigma)  # 自定义激活函数层
        self.predict = nn.Linear(n_hidden1, n_output)  # 输出层

    def forward(self, x):
        x = self.hidden1(x)
        x = self.custom1(x)
        x = self.predict(x)  # 前向传播过程
        return x

    def reset_parameters(self):
        self.hidden1.reset_parameters()
        self.predict.reset_parameters()


device = torch.device('cpu')
net = Net(n_feature=1, n_hidden1=14, n_output=1, sigma=best_sigma)
optimizer = optim.Adam(net.parameters(), lr=0.01, betas=(0.99, 0.99))
loss_func = torch.nn.MSELoss().to(device)

NJnum = 1
epoch = 15000
x_respond = torch.unsqueeze(torch.linspace(-2, 10, 300), dim=1)

train_losses = []
responses = []

# 训练循环
for num in range(NJnum):
    seed = random.sample(range(1, 10000), 1)[0]
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)

    net.reset_parameters()

    for _ in range(epoch):
        net.train()
        prediction = net(x)
        loss = loss_func(prediction, ytrain)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_losses.append(loss.item())

    net.eval()
    with torch.no_grad():
        response = net(x_respond)
        responses.append(response.numpy())

mean_train_loss = np.mean(train_losses)
mean_response = np.mean(responses, axis=0)

# 测试阶段
seeds = random.sample(range(1, 10000), 10)
test_losses = []

for seed in seeds:
    torch.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    x_test = torch.unsqueeze(torch.linspace(-2, 10, 30), dim=1).to(device)
    y_test = np.exp(-(x_test - 2) ** 2) + np.exp(-(x_test - 6) ** 2 / 10) + 1 / (x_test ** 2 + 1) + 0.12 * torch.randn(
        x_test.size()).to(device)

    net.eval()
    with torch.no_grad():
        prediction_test = net(x_test)
        test_loss = loss_func(prediction_test, y_test)
        test_losses.append(test_loss.item())

mean_test_loss = np.mean(test_losses)

print("Best hyperparameter (sigma):", best_sigma)
print("Mean Test Loss:", mean_test_loss)
print("Mean Train Loss:", mean_train_loss)

Best hyperparameter (sigma): 5.6566977648877765
Mean Test Loss: 0.01891669547185302
Mean Train Loss: 0.010154944844543934


In [21]:
# 将 x_respond 和 mean_response 保存到 CSV 文件
x_respond_np = x_respond.numpy()
output_data = np.hstack((x_respond_np, mean_response))
np.savetxt('C:/Users/Administrator/PycharmProjects/pythonProject/Forest函数激活函数方法（GELU）.csv', output_data, delimiter=',', header='x,mean_response', comments='')